# 🔧 System Diagnostic - Run This First!

Before training, we need to verify that the GPU is properly accessible. This cell will check:
1. Slurm job allocation
2. GPU hardware detection
3. CUDA environment variables
4. PyTorch CUDA compatibility
5. Common issues and solutions

**Run the diagnostic cell below FIRST before proceeding with training!**

In [ ]:
#!/usr/bin/env python3
"""
Comprehensive GPU Diagnostic for HPC Cluster
This cell diagnoses why PyTorch might show "device: cpu" even on GPU nodes
"""

import subprocess
import sys
import os

print("="*80)
print(" 🔍 COMPREHENSIVE GPU DIAGNOSTIC FOR HPC CLUSTER")
print("="*80)
print()

# ============================================================================
# 1. SLURM JOB INFORMATION
# ============================================================================
print("1️⃣  SLURM JOB ALLOCATION")
print("-" * 80)

slurm_vars = {
    'SLURM_JOB_ID': 'Job ID',
    'SLURM_JOB_NODELIST': 'Assigned Node(s)',
    'SLURM_NODEID': 'Node ID',
    'SLURM_GPUS': 'Total GPUs Allocated',
    'SLURM_GPUS_ON_NODE': 'GPUs on This Node',
    'SLURM_JOB_GPUS': 'GPU IDs Allocated',
    'SLURM_CPUS_ON_NODE': 'CPUs on Node',
    'SLURM_MEM_PER_NODE': 'Memory per Node',
}

slurm_allocated = False
for var, desc in slurm_vars.items():
    value = os.environ.get(var, 'NOT SET')
    print(f"  {desc:25s}: {value}")
    if var in ['SLURM_GPUS', 'SLURM_GPUS_ON_NODE', 'SLURM_JOB_GPUS']:
        if value != 'NOT SET' and value != '0' and value != '':
            slurm_allocated = True

print()
if slurm_allocated:
    print("  ✅ Slurm has allocated GPU(s) to this job")
else:
    print("  ⚠️  WARNING: No GPU allocation detected by Slurm!")
    print("     This job may not have requested GPU resources.")
    print()

# ============================================================================
# 2. CUDA ENVIRONMENT VARIABLES
# ============================================================================
print()
print("2️⃣  CUDA ENVIRONMENT VARIABLES")
print("-" * 80)

cuda_vars = {
    'CUDA_VISIBLE_DEVICES': 'Which GPUs are visible to CUDA',
    'CUDA_HOME': 'CUDA installation directory',
    'CUDA_PATH': 'CUDA path',
    'CUDA_ROOT': 'CUDA root directory',
    'LD_LIBRARY_PATH': 'Library path (includes CUDA libs)',
}

cuda_env_ok = False
for var, desc in cuda_vars.items():
    value = os.environ.get(var, 'NOT SET')
    if var == 'LD_LIBRARY_PATH' and value != 'NOT SET':
        # Show only CUDA-related parts
        cuda_parts = [p for p in value.split(':') if 'cuda' in p.lower() or 'CUDA' in p]
        if cuda_parts:
            print(f"  {var:25s}: {cuda_parts[0]} (and {len(cuda_parts)-1} more)")
        else:
            print(f"  {var:25s}: (no CUDA paths found)")
    else:
        print(f"  {var:25s}: {value}")
    
    if var == 'CUDA_VISIBLE_DEVICES' and value != 'NOT SET':
        cuda_env_ok = True

print()
if cuda_env_ok:
    print("  ✅ CUDA_VISIBLE_DEVICES is set")
else:
    print("  ⚠️  WARNING: CUDA_VISIBLE_DEVICES not set!")
    print("     GPUs may not be visible to applications.")
    print()

# ============================================================================
# 3. GPU HARDWARE DETECTION (nvidia-smi)
# ============================================================================
print()
print("3️⃣  GPU HARDWARE DETECTION (nvidia-smi)")
print("-" * 80)

try:
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free',
         '--format=csv'],
        capture_output=True, text=True, timeout=5
    )
    
    if result.returncode == 0:
        print(result.stdout)
        print("  ✅ GPU hardware detected successfully")
        hardware_ok = True
    else:
        print(f"  ❌ nvidia-smi failed with error:\n{result.stderr}")
        hardware_ok = False
except FileNotFoundError:
    print("  ❌ nvidia-smi command not found!")
    print("     GPU drivers may not be installed.")
    hardware_ok = False
except Exception as e:
    print(f"  ❌ Error running nvidia-smi: {e}")
    hardware_ok = False

print()

# ============================================================================
# 4. CUDA TOOLKIT VERSION
# ============================================================================
print("4️⃣  CUDA TOOLKIT VERSION")
print("-" * 80)

try:
    result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        for line in result.stdout.split('\n'):
            if 'release' in line.lower():
                print(f"  System CUDA: {line.strip()}")
                # Extract version number
                import re
                match = re.search(r'release (\d+\.\d+)', line)
                if match:
                    cuda_version = match.group(1)
                    print(f"  CUDA Version: {cuda_version}")
    else:
        print("  ⚠️  nvcc not found (CUDA toolkit may not be in PATH)")
except Exception as e:
    print(f"  ⚠️  Could not determine CUDA version: {e}")

print()

# ============================================================================
# 5. PYTORCH CUDA DETECTION
# ============================================================================
print("5️⃣  PYTORCH CUDA DETECTION")
print("-" * 80)

try:
    import torch
    print(f"  PyTorch Version: {torch.__version__}")
    print(f"  PyTorch Built with CUDA: {torch.version.cuda}")
    print(f"  CUDA Available: {torch.cuda.is_available()}")
    
    if torch.cuda.is_available():
        print(f"  CUDA Device Count: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"    GPU {i}: {torch.cuda.get_device_name(i)}")
            props = torch.cuda.get_device_properties(i)
            print(f"      Total Memory: {props.total_memory / 1024**3:.2f} GB")
        print()
        print("  ✅ PyTorch can access GPU(s)!")
        pytorch_ok = True
    else:
        print()
        print("  ❌ PyTorch CANNOT access GPU!")
        pytorch_ok = False
        
        # Diagnose why
        print()
        print("  🔍 DIAGNOSIS:")
        
        # Check CUDA version mismatch
        if torch.version.cuda:
            pytorch_cuda = torch.version.cuda
            print(f"     - PyTorch was built for CUDA {pytorch_cuda}")
            
            # Try to get system CUDA version
            try:
                nvcc_result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True, timeout=5)
                if nvcc_result.returncode == 0:
                    import re
                    match = re.search(r'release (\d+\.\d+)', nvcc_result.stdout)
                    if match:
                        system_cuda = match.group(1)
                        print(f"     - System has CUDA {system_cuda}")
                        
                        # Compare major versions
                        pytorch_major = pytorch_cuda.split('.')[0]
                        system_major = system_cuda.split('.')[0]
                        
                        if pytorch_major != system_major:
                            print()
                            print(f"     ⚠️  CUDA VERSION MISMATCH!")
                            print(f"         PyTorch needs CUDA {pytorch_major}.x")
                            print(f"         System has CUDA {system_major}.x")
                            print()
                            print(f"     💡 SOLUTION: Reinstall PyTorch with CUDA {system_major}.x support")
                            if system_major == '11':
                                print(f"         pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")
                            elif system_major == '12':
                                print(f"         pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121")
            except:
                pass
        
        # Check if CUDA_VISIBLE_DEVICES is set
        if os.environ.get('CUDA_VISIBLE_DEVICES') == '':
            print("     - CUDA_VISIBLE_DEVICES is empty (no GPUs visible)")
        elif os.environ.get('CUDA_VISIBLE_DEVICES') is None:
            print("     - CUDA_VISIBLE_DEVICES is not set")
        
        # Check if Slurm allocated GPU
        if not slurm_allocated:
            print("     - Slurm did not allocate GPU to this job")
            print("       Did you request GPU with --gres=gpu:1?")

except ImportError:
    print("  ❌ PyTorch is not installed!")
    pytorch_ok = False

print()

# ============================================================================
# 6. PYTHON ENVIRONMENT INFORMATION
# ============================================================================
print("6️⃣  PYTHON ENVIRONMENT")
print("-" * 80)

print(f"  Python Version: {sys.version}")
print(f"  Python Executable: {sys.executable}")
print(f"  Python Prefix: {sys.prefix}")
print()

# Check if we're in a virtual environment
in_venv = hasattr(sys, 'real_prefix') or (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix)
in_conda = os.path.exists(os.path.join(sys.prefix, 'conda-meta'))

if in_conda:
    print("  ✅ Running in Conda environment")
    conda_env = os.environ.get('CONDA_DEFAULT_ENV', 'unknown')
    print(f"  Conda Environment: {conda_env}")
elif in_venv:
    print("  ✅ Running in virtual environment")
else:
    print("  ⚠️  Running in system Python (not recommended)")

print()

# ============================================================================
# 7. CONDA INFORMATION (if applicable)
# ============================================================================
if in_conda:
    print("7️⃣  CONDA INFORMATION")
    print("-" * 80)
    
    try:
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            print(f"  Conda Version: {result.stdout.strip()}")
    except:
        print("  ⚠️  Could not determine conda version")
    
    try:
        result = subprocess.run(['conda', 'info', '--envs'], capture_output=True, text=True, timeout=10)
        if result.returncode == 0:
            print("\n  Available Conda Environments:")
            for line in result.stdout.split('\n'):
                if line.strip() and not line.startswith('#'):
                    print(f"    {line}")
    except:
        print("  ⚠️  Could not list conda environments")
    
    print()

# ============================================================================
# 8. INSTALLED PACKAGES DIAGNOSTICS
# ============================================================================
print("8️⃣  INSTALLED PACKAGES (KEY LIBRARIES)")
print("-" * 80)

key_packages = [
    'torch',
    'torchvision',
    'torchaudio',
    'numpy',
    'transformers',
    'diffusers',
    'accelerate',
    'datasets',
    'pillow',
    'matplotlib',
]

installed_packages = {}
missing_packages = []

for package_name in key_packages:
    try:
        if package_name == 'pillow':
            import PIL
            installed_packages[package_name] = PIL.__version__
        else:
            pkg = __import__(package_name)
            version = getattr(pkg, '__version__', 'unknown')
            installed_packages[package_name] = version
    except ImportError:
        missing_packages.append(package_name)

print("  Installed:")
for pkg, version in installed_packages.items():
    print(f"    ✅ {pkg:20s}: {version}")

if missing_packages:
    print("\n  Missing:")
    for pkg in missing_packages:
        print(f"    ❌ {pkg}")

print()

# ============================================================================
# 9. PIP PACKAGE LIST (FULL)
# ============================================================================
print("9️⃣  ALL INSTALLED PIP PACKAGES")
print("-" * 80)

try:
    result = subprocess.run([sys.executable, '-m', 'pip', 'list'], 
                          capture_output=True, text=True, timeout=30)
    if result.returncode == 0:
        lines = result.stdout.split('\n')
        # Show first 50 packages to avoid excessive output
        print("  (Showing first 50 packages, use 'pip list' to see all)")
        print()
        for i, line in enumerate(lines[:52]):  # 2 header lines + 50 packages
            if line.strip():
                print(f"  {line}")
        if len(lines) > 52:
            print(f"  ... and {len(lines) - 52} more packages")
    else:
        print("  ⚠️  Could not retrieve pip package list")
except Exception as e:
    print(f"  ⚠️  Error retrieving pip packages: {e}")

print()

# ============================================================================
# 10. PYTORCH INSTALLATION DIAGNOSTICS
# ============================================================================
print("🔟 PYTORCH INSTALLATION DIAGNOSTICS")
print("-" * 80)

if 'torch' in installed_packages:
    print("  PyTorch is installed but CUDA not available")
    print()
    print("  Possible causes:")
    print("    1. PyTorch installed without CUDA support (CPU-only version)")
    print("    2. PyTorch CUDA version doesn't match system CUDA")
    print()
    
    # Try to determine which PyTorch was installed
    try:
        result = subprocess.run(
            [sys.executable, '-m', 'pip', 'show', 'torch'],
            capture_output=True, text=True, timeout=10
        )
        if result.returncode == 0:
            print("  PyTorch installation details:")
            for line in result.stdout.split('\n'):
                if any(key in line for key in ['Version:', 'Location:', 'Requires:']):
                    print(f"    {line}")
    except:
        pass
    
    print()
    print("  💡 RECOMMENDED FIX:")
    print("     Reinstall PyTorch with CUDA 11.8 support:")
    print()
    print("     pip uninstall torch torchvision torchaudio -y")
    print("     pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")
else:
    print("  PyTorch is NOT installed")
    print()
    print("  💡 RECOMMENDED FIX:")
    print("     Install PyTorch with CUDA 11.8 support:")
    print()
    print("     pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")

print()

# ============================================================================
# 11. JUPYTER KERNEL INFORMATION
# ============================================================================
print("1️⃣1️⃣  JUPYTER KERNEL INFORMATION")
print("-" * 80)

# Get kernel name from IPython
try:
    from IPython import get_ipython
    ipython = get_ipython()
    if ipython is not None:
        # Try to get kernel info
        kernel_name = 'unknown'
        try:
            # Check if running in Jupyter
            if hasattr(ipython, 'kernel'):
                kernel_name = getattr(ipython.kernel, 'kernel_info', {}).get('name', 'unknown')
        except:
            pass
        
        # Try alternative method - check connection file
        try:
            connection_file = ipython.config.get('IPKernelApp', {}).get('connection_file', '')
            if connection_file:
                print(f"  Connection File: {os.path.basename(connection_file)}")
        except:
            pass
        
        print(f"  IPython Session: {ipython.__class__.__name__}")
    else:
        print("  IPython Session: Not running in IPython")
except Exception as e:
    print(f"  IPython Detection: {e}")

kernel_info = {
    'Jupyter Runtime': os.environ.get('JUPYTER_RUNTIME_DIR', 'NOT SET'),
    'Kernel ID': os.environ.get('KERNEL_ID', 'NOT SET'),
}

for key, value in kernel_info.items():
    print(f"  {key:20s}: {value}")

print()
print(f"  Python Interpreter: {sys.executable}")

# Check if kernel matches conda environment
if in_conda:
    conda_env = os.environ.get('CONDA_DEFAULT_ENV', 'unknown')
    kernel_python = sys.executable
    print(f"  Current Conda Env: {conda_env}")
    print(f"  Kernel Python Path: {kernel_python}")
    
    if conda_env not in kernel_python and conda_env != 'base':
        print()
        print("  ⚠️  WARNING: Jupyter kernel may not be using the correct conda environment!")
        print(f"     You may need to install ipykernel in the '{conda_env}' environment:")
        print(f"     conda activate {conda_env}")
        print(f"     conda install ipykernel")
        print(f"     python -m ipykernel install --user --name={conda_env}")

print()

# ============================================================================
# 12. SUMMARY & RECOMMENDATIONS
# ============================================================================
print("="*80)
print(" 📋 SUMMARY & RECOMMENDATIONS")
print("="*80)
print()

all_checks = {
    'Slurm GPU Allocation': slurm_allocated,
    'CUDA Environment': cuda_env_ok,
    'GPU Hardware (nvidia-smi)': hardware_ok,
    'PyTorch CUDA Access': pytorch_ok,
}

for check, status in all_checks.items():
    status_icon = "✅" if status else "❌"
    print(f"  {status_icon} {check}")

print()

if all(all_checks.values()):
    print("🎉 ALL CHECKS PASSED! GPU is ready for training.")
else:
    print("⚠️  ISSUES DETECTED. Review the diagnostic output above.")
    print()
    print("Common solutions:")
    print()
    print("1. If 'Slurm GPU Allocation' failed:")
    print("   - Make sure you started Jupyter with: bash slurm/start_jupyter.sh gpu7")
    print("   - Check job allocation: squeue -u $USER")
    print()
    print("2. If 'PyTorch CUDA Access' failed but hardware is OK:")
    print("   - Most likely PyTorch not installed or wrong CUDA version")
    print("   - See section 10 above for installation commands")
    print()
    print("3. If 'GPU Hardware' failed:")
    print("   - Job may be on a CPU-only node")
    print("   - Cancel and restart with: bash slurm/start_jupyter.sh gpu7")
    print()
    print("4. If kernel/environment mismatch:")
    print("   - See section 11 above for kernel installation commands")

print()
print("="*80)

 🔍 COMPREHENSIVE GPU DIAGNOSTIC FOR HPC CLUSTER

1️⃣  SLURM JOB ALLOCATION
--------------------------------------------------------------------------------
  Job ID                   : 3030862
  Assigned Node(s)         : gpu8
  Node ID                  : 0
  Total GPUs Allocated     : NOT SET
  GPUs on This Node        : 1
  GPU IDs Allocated        : 0
  CPUs on Node             : 2
  Memory per Node          : 16384

  ✅ Slurm has allocated GPU(s) to this job

2️⃣  CUDA ENVIRONMENT VARIABLES
--------------------------------------------------------------------------------
  CUDA_VISIBLE_DEVICES     : 0
  CUDA_HOME                : /prefix/software/CUDA/11.8.0
  CUDA_PATH                : /prefix/software/CUDA/11.8.0
  CUDA_ROOT                : /prefix/software/CUDA/11.8.0
  LD_LIBRARY_PATH          : /prefix/software/CUDA/11.8.0/nvvm/lib64 (and 2 more)

  ✅ CUDA_VISIBLE_DEVICES is set

3️⃣  GPU HARDWARE DETECTION (nvidia-smi)
---------------------------------------------------------

# 🔧 Fix PyTorch Installation

**Run this cell to fix the broken PyTorch installation.**

This will:
1. Completely remove the corrupted PyTorch installation
2. Clean pip cache
3. Install PyTorch with proper CUDA 11.8 support from the official PyTorch channel

In [ ]:
import subprocess
import sys

print("="*80)
print(" 🔧 FIXING PYTORCH INSTALLATION")
print("="*80)
print()

# Step 1: Uninstall all PyTorch packages
print("Step 1/4: Uninstalling existing PyTorch packages...")
print("-" * 80)

packages_to_remove = ['torch', 'torchvision', 'torchaudio']
for pkg in packages_to_remove:
    try:
        result = subprocess.run(
            [sys.executable, '-m', 'pip', 'uninstall', pkg, '-y'],
            capture_output=True, text=True, timeout=60
        )
        if result.returncode == 0:
            print(f"  ✅ Uninstalled {pkg}")
        else:
            print(f"  ⚠️  {pkg} was not installed or already removed")
    except Exception as e:
        print(f"  ⚠️  Error uninstalling {pkg}: {e}")

print()

# Step 2: Clean pip cache
print("Step 2/4: Cleaning pip cache...")
print("-" * 80)
try:
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'cache', 'purge'],
        capture_output=True, text=True, timeout=30
    )
    if result.returncode == 0:
        print("  ✅ Pip cache cleaned")
    else:
        print("  ⚠️  Could not clean cache (this is usually OK)")
except Exception as e:
    print(f"  ⚠️  Cache cleaning skipped: {e}")

print()

# Step 3: Install PyTorch with CUDA 11.8
print("Step 3/4: Installing PyTorch with CUDA 11.8 support...")
print("-" * 80)
print("  This may take several minutes...")
print()

try:
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'install', 
         'torch', 'torchvision', 'torchaudio',
         '--index-url', 'https://download.pytorch.org/whl/cu118'],
        capture_output=True, text=True, timeout=600  # 10 minutes max
    )
    
    if result.returncode == 0:
        print("  ✅ PyTorch installation completed!")
        # Show last few lines of output
        output_lines = result.stdout.strip().split('\n')
        print("\n  Installation log (last 10 lines):")
        for line in output_lines[-10:]:
            print(f"    {line}")
    else:
        print("  ❌ PyTorch installation FAILED!")
        print("\n  Error output:")
        print(result.stderr)
except subprocess.TimeoutExpired:
    print("  ❌ Installation timed out!")
    print("     Try running manually in terminal:")
    print("     pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")
except Exception as e:
    print(f"  ❌ Installation error: {e}")

print()

# Step 4: Verify installation
print("Step 4/4: Verifying PyTorch installation...")
print("-" * 80)

try:
    import torch
    print(f"  ✅ PyTorch imported successfully!")
    print(f"  PyTorch Version: {torch.__version__}")
    print(f"  CUDA Available: {torch.cuda.is_available()}")
    
    # Try different ways to get CUDA version (API changed in PyTorch 2.5+)
    try:
        cuda_version = torch.version.cuda
        print(f"  CUDA Version: {cuda_version}")
    except AttributeError:
        try:
            # PyTorch 2.5+ way
            import torch.cuda
            if hasattr(torch.cuda, 'get_device_properties'):
                print(f"  CUDA Runtime Version: {torch.cuda.get_arch_list()}")
        except:
            print(f"  CUDA Version: (built with CUDA support)")
    
    if torch.cuda.is_available():
        print(f"  GPU Count: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"    GPU {i}: {torch.cuda.get_device_name(i)}")
        print()
        print("  🎉 SUCCESS! PyTorch can access GPUs!")
    else:
        print()
        print("  ⚠️  WARNING: PyTorch installed but CUDA not available")
        print("     This might require restarting the Jupyter kernel")
        print("     Go to: Kernel -> Restart Kernel")
except ImportError as e:
    print(f"  ❌ PyTorch still cannot be imported!")
    print(f"     Error: {e}")
    print()
    print("  💡 Try restarting the Jupyter kernel:")
    print("     Kernel -> Restart Kernel")
except Exception as e:
    print(f"  ❌ Verification error: {e}")
    print("     However, if CUDA Available = True above, the installation worked!")

print()
print("="*80)
print()
print("📝 NEXT STEPS:")
print("   1. If installation succeeded, restart the kernel: Kernel -> Restart Kernel")
print("   2. Re-run the diagnostic cell to verify everything works")
print("   3. Proceed with the inference notebook")
print("="*80)

# 📦 Install Additional Required Packages

After PyTorch is installed, run this cell to install the remaining required packages.

In [ ]:
import subprocess
import sys

print("="*80)
print(" 📦 INSTALLING ADDITIONAL PACKAGES")
print("="*80)
print()

# List of packages to install
packages = [
    'transformers',
    'diffusers',
    'accelerate',
    'datasets',
    'matplotlib',
    'tqdm',
]

print(f"Installing {len(packages)} packages...")
print(f"Packages: {', '.join(packages)}")
print()
print("This may take a few minutes...")
print("-" * 80)

try:
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'install'] + packages,
        capture_output=True, text=True, timeout=600
    )
    
    if result.returncode == 0:
        print("✅ All packages installed successfully!")
        print()
        
        # Verify installations
        print("Verifying installations:")
        print("-" * 80)
        
        for pkg in packages:
            try:
                if pkg == 'matplotlib':
                    import matplotlib
                    version = matplotlib.__version__
                elif pkg == 'tqdm':
                    import tqdm
                    version = tqdm.__version__
                else:
                    imported_pkg = __import__(pkg)
                    version = imported_pkg.__version__
                print(f"  ✅ {pkg:20s}: {version}")
            except ImportError:
                print(f"  ❌ {pkg:20s}: Import failed")
            except AttributeError:
                print(f"  ✅ {pkg:20s}: Installed (version unknown)")
    else:
        print("❌ Installation failed!")
        print()
        print("Error output:")
        print(result.stderr)
except subprocess.TimeoutExpired:
    print("❌ Installation timed out!")
except Exception as e:
    print(f"❌ Installation error: {e}")

print()
print("="*80)
print("✅ Setup complete! You can now proceed with the inference notebook.")
print("="*80)